In [217]:
##크롤링 대상 사이트 : https://www.computer.org (IEEE)

#사용할 라이브러리

import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [218]:
#url 크롤링하는 함수 정의

def croll_link(next_url,link_name):
    req = requests.get('https://www.computer.org'+next_url)
    soup = bs(req.text, 'html.parser')
    next_r_url = soup.select(link_name)
    
    parsed = [] 
    for i in range(0,len(next_r_url)):
        parsed.append(next_r_url[i].get("href")) #url 추출해서 list에 append
    
    return parsed #list 반환

In [225]:
#크롤링 - site에서는 주제-연도-달(2-3달)-내용 으로 되어있음

# computer science subject link croll
transactionLinks = croll_link('/csdl','a.transactionLink')

# subject -> volume(year)
volumeLinks = croll_link(transactionLinks[0],'a.volumeLink')

# volume -> issue(months)
issueLinks = croll_link(volumeLinks[0],'a.issueLink')

# issue -> contents
abstractLinks = croll_link(issueLinks[0],'div.col-xs-12 > div > div > div > button > a') # contents

# abstractLinks에서 html만 추출 (javascript와 pdf도 태그가 같았기에 soup로 구분하지 못했음)
new_absLinks = []
for i in range(0,len(abstractLinks)):
    if('javascript' not in abstractLinks[i] and 'pdf' not in abstractLinks[i]):
        new_absLinks.append(abstractLinks[i])


In [234]:
abstractLinks = new_absLinks
new_absLinks = []

abstractLinks

In [236]:
#contents
req = requests.get('https://www.computer.org'+abstractLinks[1])
soup = bs(req.text, 'html.parser')

In [237]:
#title 크롤링
abs_title = (soup.select('div.abstractTitle > a > h2')[0].text) # title 추출
abs_title

'Simulating the Large-Scale Erosion of Genomic Privacy Over Time'

In [238]:
# url에 date내용이 들어있음
abs_yymm = abstractLinks[1][15:22]
abs_yymm

'2018/05'

In [246]:
#contents 크롤링
abs_contents = soup.select('div.abstractText.abstractTextMB')[0].text
abs_contents

"The dramatically decreasing costs of DNA sequencing have triggered more than a million humans to have their genotypes sequenced. Moreover, these individuals increasingly make their genomic data publicly available, thereby creating privacy threats for themselves and their relatives because of their DNA similarities. More generally, an entity that gains access to a significant fraction of sequenced genotypes might be able to infer even the genomes of unsequenced individuals. In this paper, we propose a simulation-based model for quantifying the impact of continuously sequencing and publicizing personal genomic data on a population's genomic privacy. Our simulation probabilistically models data sharing and takes into account events such as migration and interracial mating. We exemplarily instantiate our simulation with a sample population of 1,000 individuals and evaluate the privacy under multiple settings over 6,000 genomic variants and a subset of phenotype-related variants. Our findi


## Data set 늘리기


In [242]:
# Month를 다 합쳐봄 - volume은 같음
# 다 하면 너무 많을 것 같아서 일단 issue(month)만 합쳤음

total_abslink = []
for i in range(0,len(issueLinks)):
    total_abslink += croll_link(issueLinks[i],'div.col-xs-12 > div > div > div > button > a')

total_abslink

In [268]:
new_absLinks = []
for i in range(0,len(total_abslink)):
    if(total_abslink[i][-8:] == 'abs.html'):
        new_absLinks.append(total_abslink[i])
new_absLinks

['/csdl/trans/tb/2018/05/08485530-abs.html',
 '/csdl/trans/tb/2018/05/08418770-abs.html',
 '/csdl/trans/tb/2018/05/08409333-abs.html',
 '/csdl/trans/tb/2018/05/08417961-abs.html',
 '/csdl/trans/tb/2018/05/08485532-abs.html',
 '/csdl/trans/tb/2018/05/08358754-abs.html',
 '/csdl/trans/tb/2018/05/08062800-abs.html',
 '/csdl/trans/tb/2018/05/08052538-abs.html',
 '/csdl/trans/tb/2018/05/08107554-abs.html',
 '/csdl/trans/tb/2018/05/08259252-abs.html',
 '/csdl/trans/tb/2018/05/08025561-abs.html',
 '/csdl/trans/tb/2018/05/07997743-abs.html',
 '/csdl/trans/tb/2018/05/08357964-abs.html',
 '/csdl/trans/tb/2018/05/08485534-abs.html',
 '/csdl/trans/tb/2018/05/07932523-abs.html',
 '/csdl/trans/tb/2018/05/07929387-abs.html',
 '/csdl/trans/tb/2018/05/07942061-abs.html',
 '/csdl/trans/tb/2018/05/07936538-abs.html',
 '/csdl/trans/tb/2018/05/07959594-abs.html',
 '/csdl/trans/tb/2018/05/07933200-abs.html',
 '/csdl/trans/tb/2018/05/07981392-abs.html',
 '/csdl/trans/tb/2018/05/07934076-abs.html',
 '/csdl/tr

In [270]:
title =[]
date = []
contents = []

for i in range(0,len(new_absLinks)):
    req = requests.get('https://www.computer.org'+new_absLinks[i])
    soup = bs(req.text, 'html.parser')
    title.append(soup.select('div.abstractTitle > a > h2')[0].text)
    date.append(new_absLinks[i][15:22])
    contents.append(soup.select('div.abstractText.abstractTextMB')[0].text)

In [274]:
#data frame 만들기 - 총 175 rows
data = {'Title':title, 'Date':date, 'Contents':contents}

frame = pd.DataFrame(data)
frame

,Title,Date,Contents
0,GenoPri'16: International Workshop on Genome P...,2018/05,
1,Simulating the Large-Scale Erosion of Genomic ...,2018/05,The dramatically decreasing costs of DNA seque...
2,Protecting Privacy and Security of Genomic Dat...,2018/05,Re-use of patients’ health records can provide...
3,Implementation and Evaluation of an Algorithm ...,2018/05,We improve the quality of cryptographically pr...
4,Guest Editorial for Special Section on the 12t...,2018/05,
5,Identification and Analysis of Key Residues in...,2018/05,Protein–RNA complexes play important roles in ...
6,Unified Deep Learning Architecture for Modelin...,2018/05,Prediction of the spatial structure or functio...
7,Mutli-Features Prediction of Protein Translati...,2018/05,Post translational modification plays a signif...
8,Sequence-Based Prediction of Putative Transcri...,2018/05,A transcription factor (TF) is a protein that ...
9,Prediction of Hot Regions in PPIs Based on Imp...,2018/05,The hot regions in PPIs are some assembly regi...
